https://www.fukushihoken.metro.tokyo.lg.jp/hodo/saishin/hassei.html


In [1]:
import os
from WebScrapingTool import Base_UserFunction as uf
import json
import urllib

In [2]:
# https://www.shibutan-bloomers.com/python_library_pdfminer-six/2124/#21PDFJupyterNotebook

from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
from io import StringIO
import jaconv
from datetime import date
from japanera import Japanera, EraDate
import unicodedata
import copy

janera = Japanera()

def parse(filePath, type):
    print("parse start : " + filePath)
    getText = ''
    retData = '{'
    try:
        with open(filePath, 'rb') as fp:
            outfp = StringIO()

            rmgr = PDFResourceManager()
            lprms = LAParams()
            device = TextConverter(rmgr, outfp, laparams=lprms)
            iprtr = PDFPageInterpreter(rmgr, device)

            for page in PDFPage.get_pages(fp):
                iprtr.process_page(page)
            getText = outfp.getvalue()
            device.close()
            outfp.close()

    except:
        print("file open error...")
        return ""

    baseList = getText.split()

    idx = -1
    modeSokuhou = (
        '患者の発生状況', 
        '行政検査数', 
        '都の独自検査件数', 
        '10歳未満', 
        '都内発生数', 
        '区市町村別患者数' 
    )

    modeAdd = (
        '患者の発生状況', 
        '都内発生数', 
        '重症者の属性',
        '入院・療養等調整中の陽性患者について' 
    )


    endWord = '都内コロナウイルス陽性患者'

    mode = list()
    isFitPDF = False
    for l in baseList:
        if '別紙' in l or '総数' in l:
            retData += '"isAddData" : "False", ' 
            mode = copy.deepcopy(modeSokuhou)
            isFitPDF = True
            break
        elif '追加' in l:
            retData += '"isAddData" : "True", ' 
            mode = copy.deepcopy(modeAdd)
            isFitPDF = True
            break
        elif endWord in l:
            break
    
    if not isFitPDF:
        print("miss match header..")
        return ""

    # date
    _idx = 1
    for l in baseList:
        if '◆令和' in l:
            _date = jaconv.z2h(l, kana=False, ascii=False, digit=True)
            _date = _date.replace('◆', '')
            _date = _date[:_date.find('日') + 1]
            _date = janera.strptime(_date, "%-E%-kO年%-km月%-kd日")
            _date = _date[0].strftime('%Y/%m/%d')
            retData += '"date" : "' + _date
            break
        else:
            _idx += 1
    
    for l in baseList:
        if endWord in l:
            break

        if idx < len(mode) - 1 and mode[idx + 1] in l:
            idx += 1
            retData += '", "' + mode[idx] + '" : "'
        
        tmp = l.replace('(', '').replace(')', '').replace(',', '').replace('%', '')
        if tmp.encode('utf-8').isdigit():
            retData += tmp + ","
        elif tmp.isascii() and '.' in tmp:
            retData += tmp + ","

    retData += '"}'
    #print(retData)
    return retData


In [3]:
def main():
    print("\n[Start]"  + uf.getNowTime() + '\n')

    #設定ファイルから必要な情報を取得する
    #タグ
    tag_debug = '[a]'
    tag_saveFolder = '[b]'
    tag_loadFileName = '[c]'
    tag_saveFileName = '[j]'
    
    isDebug = False
    _saveFolder = ''
    _loadFileName = ''
    _saveFileName = ''
    
    try:
        with open('_Setting.txt', mode='r') as f:
            lines = f.readlines()
            for l in lines:
                if l.startswith(tag_debug, 0, 3):
                    if (l.replace(tag_debug, '').rstrip()).lower() == 'true':
                        isDebug = True
                    else:
                        isDebug = False                    

                if l.startswith(tag_saveFolder, 0, 3):
                    _saveFolder = l.replace(tag_saveFolder, '').rstrip()
             
                if l.startswith(tag_loadFileName, 0, 3):
                    _loadFileName = l.replace(tag_loadFileName, '').rstrip()
             
                if l.startswith(tag_saveFileName, 0, 3):
                    _saveFileName = l.replace(tag_saveFileName, '').rstrip()
             
    except:
        print('[!!!ERROR!!!] Read Setting.text')
        return        
    
    if len(_saveFolder) <= 0:
        print('[!!!ERROR!!!] Image data storage folder is None!')
        return  
    
    baseFile =_saveFolder + "/" + _loadFileName
    print(baseFile)

    saveFile =_saveFolder + "/" + _saveFileName
    print(saveFile)
    
    # ファイルを開く
    updateList = list()
    parseList = list()
    with open(baseFile, mode='r') as f:
        cnt = 0
        for line in f:
            l = line
            j = json.loads(line)
            type = j['type']
            fileName = j['name']
            isParse = j['isParse']
            if isParse == "False":
                _data = parse(_saveFolder + "/" + fileName, type)
                if len(_data) <= 0:
                    continue
                parseList.append(_data + "\n")
                cnt += 1
#                print("...Parse PDF : " + fileName + '  ' + str(cnt))
                l = l.replace('"isParse" : "False"', '"isParse" : "True"')
            updateList.append(l)

        print('\n...Get Size :' + str(cnt) + '\n')

    # ファイル更新
    with open(baseFile, mode='w') as f:
        for line in updateList:
                uf.fileWrite(f, line)

    # パースしたデータを追加
    with open(saveFile, mode='a') as f:
        for line in parseList:
                uf.fileWrite(f, line)
    # 重複データ削除
    uf.fileDataSlim(saveFile) 

    print("\n[ End ]"  + uf.getNowTime() + '\n')
    
    
if __name__ == '__main__':
    main()


[Start]20210731002940

#data/dataList.json
#data/parsePDF.json
parse start : #data/corona2275.files2275-2.pdf
（情報提供）新型コロナウイルスに関連した患者の発生について【追加情報】
parse start : #data/20210723_0723-01-01.pdf
別紙
parse start : #data/20210617_0617-30-02.pdf
（情報提供）新型コロナウイルスに関連した患者の発生について【追加情報】
parse start : #data/20200417_20200417.pdf
総数
parse start : #data/20201023_20201023.pdf
別紙
parse start : #data/20200420_11_01.pdf
総数
parse start : #data/20210601_0601-20-01.pdf
別紙
parse start : #data/20200329_133_02.pdf
都内コロナウイルス陽性患者
miss match header..
parse start : #data/20201220_20201219.pdf
別紙
parse start : #data/20200513_0513-12.pdf
◆令和２年５月１３日１８時３０分時点
miss match header..
parse start : #data/20201114_04_00.pdf
別紙
parse start : #data/20210605_01_01.pdf
（情報提供）新型コロナウイルスに関連した患者の発生について【追加情報】
parse start : #data/20201026_20201025.pdf
別紙
parse start : #data/20210724_0724-01-02.pdf
（情報提供）新型コロナウイルスに関連した患者の発生について【追加情報】
parse start : #data/20200414_20_02.pdf
総数
parse start : #data/20200623_11_00.pdf
◆令和２年６月２３日１８時００分時点
miss m